In [1]:
import json
import psycopg2
import pandas as pd

In [2]:
with open('jobs_data.json', encoding= 'utf-8') as f:
    oferty = json.load(f)

In [3]:
oferty[1]

{'Źródło': 'justjoin.it',
 'Link': 'https://justjoin.it/offers/haddad-brands-europe-power-bi-developer-partly-in-paris-krakow',
 'Pozycja': 'Power BI Developer (Partly in Paris)',
 'Firma': 'Haddad Brands Europe',
 'Min salary': 20000.0,
 'Max salary': 27000.0,
 'Currency': 'PLN',
 'Skills': ['Team Player',
  'English',
  'Power BI',
  'MS SQL',
  'Proactivity',
  'Critical thinking',
  'Software Architecture',
  'French'],
 'Category': 'Data',
 'Seniority': 'Senior',
 'Adres': None,
 'Podsumowanie': 'This includes adding charts, maps, pivot tables, filters, KPI visualizations, etc.• Designs visual reports based on the needs of HBE or end users. These reports can be static or interactive, allowing users to derive meaningful insights from the data.• Publishes, after validation, reports online, allowing users to view and interact with them via a web browser or mobile application.• Also configures appropriate access permissions to control who can view and interact with the data.• Ensures 

In [4]:
def create_tables(connection):
    cursor = connection.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DIMFirma (
        ID SERIAL PRIMARY KEY,
        Nazwa VARCHAR(100),
        Adres VARCHAR(100)
    )
    ''')
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DIMStanowisko (
        ID SERIAL PRIMARY KEY,
        Nazwa VARCHAR(100)
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DIMUmiejętność (
        ID SERIAL PRIMARY KEY,
        Nazwa VARCHAR(100)
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DIMWaluta (
        ID SERIAL PRIMARY KEY,
        Nazwa VARCHAR(3)
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DIMŹródło (
        ID SERIAL PRIMARY KEY,
        Nazwa VARCHAR(100)
    )
    ''')
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DIMKategoria (
        ID SERIAL PRIMARY KEY,
        Nazwa VARCHAR(100)
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS FACTOferta (
        ID SERIAL PRIMARY KEY,
        ID_Firmy INTEGER,
        ID_Stanowiska INTEGER,
        ID_Kategorii INTEGER,
        ID_Waluty INTEGER,
        ID_Źródła INTEGER,
        Link VARCHAR(200),
        Umiejętności VARCHAR(200),
        Seniority VARCHAR(200),
        Wynagrodzenie_MIN INTEGER,
        Wynagrodzenie_MAX INTEGER,
        Podsumowanie TEXT,
        FOREIGN KEY (ID_Firmy) REFERENCES DIMFirma(ID),
        FOREIGN KEY (ID_Stanowiska) REFERENCES DIMStanowisko(ID),
        FOREIGN KEY (ID_Kategorii) REFERENCES DIMKategoria(ID),
        FOREIGN KEY (ID_Waluty) REFERENCES DIMWaluta(ID),
        FOREIGN KEY (ID_Źródła) REFERENCES DIMŹródło(ID)
    )
    ''')
    connection.commit()
    cursor.close()

In [5]:
def get_or_insert_firma(connection, nazwa, adres):
    cursor = connection.cursor()
    cursor.execute("SELECT ID FROM DIMFirma WHERE Nazwa = %s", (nazwa,))
    result = cursor.fetchone()
    if result is None:
        cursor.execute("INSERT INTO DIMFirma (Nazwa, Adres) VALUES (%s, %s) RETURNING ID", (nazwa, adres))
        result = cursor.fetchone()
    connection.commit()
    cursor.close()
    return result[0]

In [6]:
def get_or_insert_stanowisko(connection, nazwa):
    cursor = connection.cursor()
    cursor.execute("SELECT ID FROM DIMStanowisko WHERE Nazwa = %s", (nazwa,))
    result = cursor.fetchone()
    if result is None:
        cursor.execute("INSERT INTO DIMStanowisko (Nazwa) VALUES (%s) RETURNING ID", (nazwa,))
        result = cursor.fetchone()
    connection.commit()
    cursor.close()
    return result[0]

In [7]:
def get_or_insert_kategoria(connection, nazwa):
    cursor = connection.cursor()
    cursor.execute("SELECT ID FROM DIMKategoria WHERE Nazwa = %s", (nazwa,))
    result = cursor.fetchone()
    if result is None:
        cursor.execute("INSERT INTO DIMKategoria (Nazwa) VALUES (%s) RETURNING ID", (nazwa,))
        result = cursor.fetchone()
    connection.commit()
    cursor.close()
    return result[0]

In [8]:
def get_or_insert_waluta(connection, nazwa):
    cursor = connection.cursor()
    cursor.execute("SELECT ID FROM DIMWaluta WHERE Nazwa = %s", (nazwa,))
    result = cursor.fetchone()
    if result is None:
        cursor.execute("INSERT INTO DIMWaluta (Nazwa) VALUES (%s) RETURNING ID", (nazwa,))
        result = cursor.fetchone()
    connection.commit()
    cursor.close()
    return result[0]

In [9]:
def get_or_insert_zrodlo(connection, nazwa):
    cursor = connection.cursor()
    cursor.execute("SELECT ID FROM DIMŹródło WHERE Nazwa = %s", (nazwa,))
    result = cursor.fetchone()
    if result is None:
        cursor.execute("INSERT INTO DIMŹródło (Nazwa) VALUES (%s) RETURNING ID", (nazwa,))
        result = cursor.fetchone()
    connection.commit()
    cursor.close()
    return result[0]

In [10]:
def get_or_insert_umiejetnosc(connection, nazwa):
    cursor = connection.cursor()
    cursor.execute("SELECT ID FROM DIMUmiejętność WHERE Nazwa = %s", (nazwa,))
    result = cursor.fetchone()
    if result is None:
        cursor.execute("INSERT INTO DIMUmiejętność (Nazwa) VALUES (%s) RETURNING ID", (nazwa,))
        result = cursor.fetchone()
    connection.commit()
    cursor.close()
    return result[0]

In [11]:
def insert_oferta(connection, firma, stanowisko, kategoria, waluta, zrodlo, link, umiejetnosci, seniority, wynagrodzenie_min, wynagrodzenie_max, podsumowanie):
    cursor = connection.cursor()

    id_firmy = get_or_insert_firma(connection, firma, None)
    id_stanowiska = get_or_insert_stanowisko(connection, stanowisko)
    id_kategorii = get_or_insert_kategoria(connection, kategoria)
    id_waluty = get_or_insert_waluta(connection, waluta)
    id_zrodla = get_or_insert_zrodlo(connection, zrodlo)

    umiejetnosci_ids = [get_or_insert_umiejetnosc(connection, umiejetnosc) for umiejetnosc in umiejetnosci]
    umiejetnosci_str = ', '.join(map(str, umiejetnosci_ids))

    insert_query = '''
    INSERT INTO FACTOferta (
        ID_Firmy, ID_Stanowiska, ID_Kategorii, ID_Waluty, ID_Źródła,
        Link, Umiejętności, Seniority, Wynagrodzenie_MIN, Wynagrodzenie_MAX, Podsumowanie
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''
    cursor.execute(insert_query, (id_firmy, id_stanowiska, id_kategorii, id_waluty, id_zrodla, link, umiejetnosci_str, seniority, wynagrodzenie_min, wynagrodzenie_max, podsumowanie))
    connection.commit()
    cursor.close()

In [12]:
def fetch_factoferta(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM FACTOferta LIMIT 5")
    rows = cursor.fetchall()
    cursor.close()
    return rows

In [13]:
try:
    connection = psycopg2.connect(
        dbname = 'projekt',
        user = 'postgres',
        password = 'postgres123',
        host = 'localhost',
        port = '5432',
        options = "-c search_path=public",
        client_encoding = "UTF8"
    )
    print("Połączono z bazą danych")
    create_tables(connection)
    for oferta in oferty:
        insert_oferta(
            connection,
            firma = oferta['Firma'],
            stanowisko = oferta['Pozycja'],
            kategoria = oferta['Category'],
            waluta = oferta['Currency'],
            zrodlo = oferta['Źródło'],
            link = oferta['Link'],
            umiejetnosci = oferta['Skills'],
            seniority = oferta['Seniority'],
            wynagrodzenie_min = int(oferta['Min salary']),
            wynagrodzenie_max = int(oferta['Max salary']),
            podsumowanie = oferta['Podsumowanie']
        )
    rows = fetch_factoferta(connection)
except Exception as error:
    print(f"Błąd łaczenia z bazą danych: {error}")
    
finally:
    if connection:
        connection.close()
        print("Zamknięto połączenie z bazą danych")

Połączono z bazą danych
Zamknięto połączenie z bazą danych


In [14]:
for row in rows:
    print(row)

(1, 1, 1, 1, 1, 1, 'https://justjoin.it/offers/cloudfide-business-intelligence-power-bi-developer-krakow-386118', '1, 2, 3, 4, 5, 6, 7, 8, 9', 'Senior', 11500, 25000, "Creative and curious, you thrive on using data to fuel smart decisions. The role primarily involves translating business logic into DAX, optimizing user-written measures, and training users for enhanced self-sufficiency. The project offers an opportunity to improve problem-solving skills and gain substantial experience in data modeling.YOUR IMPACT ZONE Translating business requirements into technical solutions. 3+ years of experience in delivering complex BI solutions - it's your time to excel! Familiarity with public cloud architecture, security, networking concepts (MS Azure preferred) - we like our clouds secure and efficient. GROWTH:\xa0Skyrocket your career by exploring new territories - you can work on various projects related to Big Data and Cloud. COLLABORATION:\xa0Be part of our diverse, passionate team, where e